In [ ]:
from io import StringIO
import os 
from dotenv import load_dotenv

import climpred
import xarray as xr
import xesmf as xe
import numpy as np
import pandas as pd
import regionmask
import geopandas as gp
from climpred import HindcastEnsemble
from datetime import datetime

import xhistogram.xarray as xhist
from sklearn.metrics import roc_auc_score
import pandas as pd

In [ ]:
load_dotenv()

data_path=os.getenv("data_path")

## The Forecast Evaluation Process

Following process is invovled in assessing the accuracy of drought forecasts relative to observed data. 

1. **Observation and Forecast Data**: The process begins with two primary datasets - observed data (`obs_data`) and ensemble forecast data (`ens_data`).

2. **Threshold Determination**: A predefined threshold is used to classify whether a drought event occurred (based on observed and forecasted data). This is the threshold value for different season as discussed in the paper. The used threshold values are indicated in the function `get_threshold`.

3. **Dichotomous Events Creation**:
   - **Observed Events**: An event is flagged in the observed data (`obs_event1`) if the observed value is less than or equal to the threshold, indicating the occurrence of a drought. The less than threshold value is following Gabriela et.al 2023. 
   - **Forecasted Events**: Instead of averaging ensemble forecasts, an empirical probability is calculated. This reflects the proportion of ensemble members predicting values at or below the given threshold.

4. **Probability Threshold for Forecasts**: The forecast data is then classified into dichotomous events based on a trigger value. This step converts the forecast probability into a binary outcome - whether a drought is forecasted to occur or not. In the process, all the possible trigger value between 0 to 1 is subject to testing as in the line `trigger_values = xr.DataArray(np.linspace(0, 1, num=100), dims=['trigger_value'])`. 

5. **Contingency Table Construction**: Using `xhist.histogram`, a 2D histogram (contingency table) is created from the binary observed and forecasted events. This table quantifies the relationship between observed occurrences/non-occurrences and forecasted occurrences/non-occurrences of drought.

6. **Skill Score Calculation**: From the contingency table, various skill scores are calculated, including hit rates, false alarm ratios, and others. These metrics provide a comprehensive view of forecast accuracy and reliability.

7. **AUROC Calculation**: The Area Under the Receiver Operating Characteristic (AUROC) score is calculated, offering a measure of the forecast's ability to discriminate between the occurrence and non-occurrence of drought events. Bootstrap method was used and it is the expensive step, which one of the reason to hinders to do year wise score calcualtion. 

8. **Final Output**: The process culminates in the generation of a DataFrame summarizing the calculated skill scores for various trigger values, which is then saved to a CSV file. This summary facilitates the evaluation of forecast performance across different probability thresholds.

9. **Lack of Yearly Variability Consideration**: The method evaluates forecasts against observations over the entire 43-year period without segregating by individual years. This could mask interannual variability or trends in forecast skill. However the bootstrap compute cost along with exponantial increase in comparision hinders to do, need further discussion. 

---


## The summary of steps are as follows

1. **Load Datasets**: Based on the provided `season_str` length, either SPI3 or SPI4 datasets are loaded for both forecast (`kn_fct`) and observed (`kn_obs`) data from a specified `data_path`.

2. **Generate Region Masks**: Utilizes `ken_mask_creator()` to create masks for specified regions or districts using the `regionmask` library, facilitating region-specific analyses.

3. **Select Region**: Extracts geographical bounds from a combined GeoDataFrame and selects forecast and observation data within these bounds for the specified `region_id`.

4. **Initialize Hindcast Ensemble**: Creates a `HindcastEnsemble` object with the selected forecast data and adds the corresponding observations to it.

5. **Subset for Lead Time**: Subsets the forecast data for the given `lead_int`, ensuring analysis is conducted at the specified forecast lead time.

6. **Generate Seasonal Product Names**: Depending on `season_str` length, either `spi3_prod_name_creator` or `spi4_prod_`name_creator` is called to generate a list of seasonal product names for both forecast and observed data, facilitating season-specific filtering.

7. **Assign and Filter by Season**: Assigns generated seasonal product names as coordinates to both datasets and filters them to include only data corresponding to the specified `season_str`.

8. **Align Time Coordinates**: Aligns the observed dataset time coordinates with the forecast dataset's valid time coordinates, ensuring that both datasets are comparable in time for analysis.

9. **Drop NaNs and Subset Data**: Drops any NaN values that may have resulted from the reindexing process and subsets the data to include only the relevant SPI variable (`spi3` or `spi4`), based on `season_str` length.

10. **Calculate Scores**: The `score_gen` function then takes these prepared datasets along with the `region_id`, `season_str`, and `level` to calculate various verification scores and saves the results to a CSV file.

This process ensures that the observed and forecasted datasets are correctly prepared and aligned for a specified region, season, and lead time, allowing for the accurate calculation of verification scores such as hit rates, false alarm ratios, and AUROC scores.

In [ ]:


def ken_mask_creator():
    """
    Utiliity for generating region/district masks using regionmask library

    Returns
    -------
    the_mask : TYPE
        DESCRIPTION.
    rl_dict : TYPE
        DESCRIPTION.

    """
    dis=gp.read_file(f'{data_path}Karamoja_boundary_dissolved.shp')
    mbt_path=os.getenv("mbt_path")
    reg=gp.read_file(f'{data_path}wajir_mbt_extent.shp')
    mds=pd.concat([dis,reg])
    mds1=mds.reset_index()
    mds1['region']=[0,1,2]
    mds1['region_name']=['Karamoja', 'Marsabit','Wajir']
    mds2=mds1[['geometry','region','region_name']]
    rl_dict=dict(zip(mds2.region, mds2.region_name))
    the_mask = regionmask.from_geopandas(mds2,numbers='region',overlap=True)
    return the_mask, rl_dict, mds2

def spi3_prod_name_creator(ds_ens,var_name):
    """
    Convenience function to generate a list of SPI product
    names, such as MAM, so that can be used to filter the 
    SPI product from dataframe

    added with method to convert the valid_time in CF format into datetime at
    line 3, which is the format given by climpred valid_time calculation 

    Parameters
    ----------
    ds_ens : xarray dataframe
        The data farme with SPI output organized for 
        the period 1981-2023.

    Returns
    -------
    spi_prod_list : String list
        List of names with iteration of SPI3 product names such as
        ['JFM','FMA','MAM',......]

    """
    db=pd.DataFrame()
    db['dt']=ds_ens[var_name].values
    db['dt1'] = db['dt'].apply(lambda x: datetime(x.year, x.month, x.day,
                                                                     x.hour, x.minute, x.second))
    #db['dt1']=db['dt'].to_datetimeindex()
    db['month']=db['dt1'].dt.strftime('%b').astype(str).str[0]
    db['year']=db['dt1'].dt.strftime('%Y')
    db['spi_prod'] = db.groupby('year')['month'].shift(2)+db.groupby('year')['month'].shift(1) + db.groupby('year')['month'].shift(0)
    spi_prod_list=db['spi_prod'].tolist()
    return spi_prod_list


def spi4_prod_name_creator(ds_ens,var_name):
    """
    Convenience function to generate a list of SPI product
    names, such as MAM, so that can be used to filter the 
    SPI product from dataframe

    added with method to convert the valid_time in CF format into datetime at
    line 3, which is the format given by climpred valid_time calculation 

    Parameters
    ----------
    ds_ens : xarray dataframe
        The data farme with SPI output organized for 
        the period 1981-2023.

    Returns
    -------
    spi_prod_list : String list
        List of names with iteration of SPI3 product names such as
        ['JFM','FMA','MAM',......]

    """
    db=pd.DataFrame()
    db['dt']=ds_ens[var_name].values
    db['dt1'] = db['dt'].apply(lambda x: datetime(x.year, x.month, x.day,
                                                                     x.hour, x.minute, x.second))
    #db['dt1']=db['dt'].to_datetimeindex()
    db['month']=db['dt1'].dt.strftime('%b').astype(str).str[0]
    db['year']=db['dt1'].dt.strftime('%Y')
    db['spi_prod'] = db.groupby('year')['month'].shift(3)+db.groupby('year')['month'].shift(2)+db.groupby('year')['month'].shift(1) + db.groupby('year')['month'].shift(0)
    spi_prod_list=db['spi_prod'].tolist()
    return spi_prod_list


def make_obs_fct_dataset(region_id,season_str,lead_int):
    """
    Prepares observed and forecasted dataset subsets for a specific region, season, and lead time.

    This function loads observed and forecasted datasets based on the season string length (indicating SPI3 or SPI4),
    applies regional masking, selects the data for the given region by its ID, and subsets the data for the specified
    season and lead time. It then aligns the observed dataset time coordinates with the forecasted dataset valid time
    coordinates and returns both datasets.

    Parameters:
    - region_id (int): The identifier for the region of interest.
    - season_str (str): A string representing the season. The length of this string determines whether SPI3 or SPI4
                        datasets are used ('mam', 'jjas', etc. for SPI3, and longer strings for SPI4).
    - lead_int (int): The lead time index for which the forecast dataset is to be subset.

    Returns:
    - obs_data (xarray.DataArray): The subsetted observed data array for the specified region, season, and aligned time coordinates.
    - ens_data (xarray.DataArray): The subsetted forecast data array for the specified region, season, lead time, and aligned time coordinates.

    Notes:
    - The function assumes the existence of a `data_path` variable that specifies the base path to the dataset files.
    - It requires the `xarray` library for data manipulation and assumes specific naming conventions for the dataset files.
    - Regional masking and season-specific processing rely on externally defined functions and naming conventions.
    - The final alignment of observed dataset time coordinates with forecasted dataset valid time coordinates ensures
      comparability between observed and forecasted values for verification purposes.

    Example Usage:
    >>> obs_data, ens_data = make_obs_fct_dataset(1, 'mam', 0)
    >>> print(obs_data)
    >>> print(ens_data)

    This would load the observed and forecasted SPI3 datasets for region 1 during the 'mam' season and subset them
    for lead time index 0, aligning the observed data time coordinates with the forecasted data valid time coordinates.
    """
    if len(season_str) == 3:
        kn_fct=xr.open_dataset(f'{data_path}kn_fct_spi3.nc')
        kn_obs=xr.open_dataset(f'{data_path}kn_obs_spi3.nc')
    else:
        kn_fct=xr.open_dataset(f'{data_path}kn_fct_spi4.nc')
        kn_obs=xr.open_dataset(f'{data_path}kn_obs_spi4.nc')
    the_mask, rl_dict,mds1=ken_mask_creator()
    bounds = mds1.bounds
    #bounds.iloc[0].minx
    llon=bounds.iloc[region_id].minx
    llat=bounds.iloc[region_id].miny
    ulon=bounds.iloc[region_id].maxx
    ulat=bounds.iloc[region_id].maxy
    a_fc=kn_fct.sel(lon=slice(llon, ulon), lat=slice(llat,ulat))
    a_obs=kn_obs.sel(lon=slice(llon, ulon), lat=slice(llat,ulat))
    hindcast = HindcastEnsemble(a_fc)
    hindcast = hindcast.add_observations(a_obs)
    #hindcast
    #spi_cdb1spi3_prod_name_creator(ds_ens)
    a_fc1=hindcast.get_initialized()
    a_fc2=a_fc1.isel(lead=lead_int)
    if len(season_str) == 3:
        spi_prod_list=spi3_prod_name_creator(a_fc2,'valid_time')
        obs_spi_prod_list=spi3_prod_name_creator(a_obs,'time')
    else:
        spi_prod_list=spi4_prod_name_creator(a_fc2,'valid_time')
        obs_spi_prod_list=spi4_prod_name_creator(a_obs,'time')
    a_fc2 = a_fc2.assign_coords(spi_prod=('init',spi_prod_list))
    a_fc3=a_fc2.where(a_fc2.spi_prod==season_str, drop=True)
    #obsertations
    a_obs1 = a_obs.assign_coords(spi_prod=('time',obs_spi_prod_list))
    a_obs2=a_obs1.where(a_obs1.spi_prod==season_str, drop=True)
    #valid_time_series = a_fc3.valid_time.to_series().reset_index(drop=True).drop_duplicates()
    valid_time_flattened = a_fc2.valid_time.to_dataframe().reset_index().drop_duplicates(subset='valid_time')['valid_time']
    valid_time_flattened.columns=['valid_time','cc']
    #valid_time_flattened['valid_time'] = pd.to_datetime(valid_time_flattened['valid_time'])
    # Apply lambda function to create 'dt1' column
    #valid_time_flattened['dt1'] = valid_time_flattened['valid_time'].apply(
    #    lambda x: datetime(x.year, x.month, x.day, x.hour, x.minute, x.second)
    #)
    #
    valid_time_flattened['dt1'] =valid_time_flattened['valid_time'].apply(lambda x: datetime(x.year, x.month, x.day,x.hour, x.minute, x.second))
    # Ensure the valid_time is in 'YYYY-MM-DD' string format
    #valid_time_flattened['dt2'] = valid_time_flattened['dt1'].dt.strftime('%Y-%m-%d')
    valid_time_flattened['dt1'] = valid_time_flattened['dt1'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')
    valid_time_flattened['dt1'] = pd.to_datetime(valid_time_flattened['dt1'])
    # Convert to xarray DataArray with time as the dimension name
    #valid_time_da = xr.DataArray(valid_time_flattened['dt1'], dims=['time'])
    valid_time_da = xr.DataArray(valid_time_flattened['dt1'], dims=['time'],coords=valid_time_flattened['dt1'])
    a_obs3 = a_obs2.reindex(time=valid_time_da)
    #a_obs4 = a_obs3.reindex(time=a_obs2.time)
    #a_obs4 = a_obs3.sel(time=a_obs2.time, drop=True)
    a_obs3 = a_obs3.dropna(dim='time')
    if len(season_str) == 3:
        obs_data=a_obs3['spi3']
        ens_data=a_fc3['spi3']
    else:
        obs_data=a_obs3['spi4']
        ens_data=a_fc3['spi4']
    return obs_data, ens_data


# Calculate AUROC using bootstrap
def calculate_auroc(hits, misses, false_alarms, correct_negatives):
    """
    Calculates the Area Under the Receiver Operating Characteristic (AUROC) curve for a set of forecasts relative to observations.

    This function computes the AUROC score as a measure of the forecast's ability to discriminate between two classes:
    events that occurred (drought) and events that did not occur (no drought). The AUROC score ranges from 0 to 1,
    where a score of 0.5 suggests no discriminative ability (equivalent to random chance), and a score of 1 indicates perfect discrimination.

    Parameters:
    - hits (int): The number of correctly forecasted events (true positives).
    - misses (int): The number of events that were observed but not forecasted (false negatives).
    - false_alarms (int): The number of non-events that were incorrectly forecasted as events (false positives).
    - correct_negatives (int): The number of non-events that were correctly forecasted (true negatives).

    Returns:
    - auroc (float): The calculated AUROC score for the given contingency table values.

    Note:
    - This function is designed to work with binary classification problems, such as predicting the occurrence or non-occurrence of drought events.
    - It requires the `roc_auc_score` function from the `sklearn.metrics` module and `numpy` for handling arrays.

    Example usage:
    >>> auroc_score = calculate_auroc(50, 30, 20, 100)
    >>> print(f"AUROC Score: {auroc_score}")
    """
    total_positives = hits + misses
    total_negatives = correct_negatives + false_alarms
    y_true = np.concatenate((np.ones(total_positives), np.zeros(total_negatives)))
    y_scores = np.concatenate((np.ones(hits), np.zeros(misses + false_alarms + correct_negatives)))
    auroc = roc_auc_score(y_true, y_scores)
    return auroc

# Define the event threshold
#threshold = -0.14


def get_threshold(region_id, season, level):
    """
    Retrieves the drought threshold value for a specified region, season, and drought level.

    The function reads predefined threshold values from a CSV-format string. It looks up the threshold for the given
    region ID, season, and drought level ('mod' for moderate, 'sev' for severe, or 'ext' for extreme). These thresholds
    are specific to certain regions and seasons and indicate the level at which a drought event of a particular severity
    is considered to occur.

    Parameters:
    - region_id (int): The integer identifier for the region of interest.
    - season (str): The season for which the threshold is required. Expected values are season codes such as 'mam' (March-April-May),
                    'jjas' (June-July-August-September), 'ond' (October-November-December), etc.
    - level (str): The drought severity level for which the threshold is requested. Valid options are 'mod' for moderate,
                   'sev' for severe, and 'ext' for extreme drought conditions.

    Returns:
    - float: The threshold value for the specified region, season, and drought level. Returns None if no threshold is found for the given inputs.

    Note:
    - This function uses a hardcoded CSV string as its data source. In a production environment, it's recommended to
      store and retrieve such data from a more robust data management system.
    - The function requires the pandas library for data manipulation and the StringIO module from io for string-based data input.

    Example usage:
    >>> threshold = get_threshold(1, 'mam', 'mod')
    >>> print(threshold)
    -0.14
    """
    data = """region_id,region,season,mod,sev,ext
    0,kmj,mam,-0.03,-0.56,-0.99
    0,kmj,jjas,-0.01,-0.41,-0.99
    1,mbt,mam,-0.14,-0.38,-0.8
    1,mbt,ond,-0.15,-0.53,-0.71
    2,wjr,mam,-0.19,-0.45,-0.75
    2,wjr,ond,-0.29,-0.76,-0.9
    """
    # Use StringIO to convert the string data to a file-like object
    data_io = StringIO(data)
    # Read the data into a pandas DataFrame
    df = pd.read_csv(data_io)
    thresholds_dict = { (row['region_id'], row['season']): {'mod': row['mod'], 'sev': row['sev'], 'ext': row['ext']}
                   for _, row in df.iterrows() }
    # Retrieve the dictionary for the given region_id and season
    season_thresholds = thresholds_dict.get((region_id, season), {})
    # Return the threshold for the given level (mod, sev, ext), or None if not found
    return season_thresholds.get(level)


def score_gen(obs_data,ens_data,lead_int,region_id,season_str,level):
    """
    Generates scores for evaluating the performance of drought forecasts based on observed data and ensemble forecast data.

    Parameters:
    - obs_data (xarray.DataArray): The observed data array.
    - ens_data (xarray.DataArray): The ensemble forecast data array.
    - lead_int (int): Lead time integer indicating the forecast lead time.
    - region_id (str): A string identifier for the region of interest.
    - season_str (str): A string representing the season (e.g., 'JJA' for June-July-August).
    - level (int/float): The threshold level to define a drought event.

    The function processes the observed and forecast data to compute various skill scores and area under the ROC curve (AUROC) 
    scores for different trigger values ranging from 0 to 1. These scores include hit rates, false alarm ratios, bias scores, Hanssen and 
    Kuipers scores, Heidke skill scores, and AUROC scores along with their confidence intervals.

    The results are saved to a CSV file named with the pattern 'regionid_season_level_ltleadtime.csv' in a specified data path, and also returned as a pandas DataFrame.

    Returns:
    - df (pandas.DataFrame): A DataFrame containing the computed scores for each trigger value along with the threshold, hit rates, false alarm ratios,
    bias scores, Hanssen and Kuipers scores, Heidke skill scores, AUROC scores, and AUROC confidence intervals.
    
    Note:
    - This function requires the xarray, numpy, pandas, and xhistogram libraries.
    - Ensure that `data_path` variable is defined in your environment to specify the output directory for the CSV file.
    """
    sc_season_str=season_str.lower()
    threshold=get_threshold(region_id, sc_season_str, level)
    # Define the trigger values
    trigger_values = xr.DataArray(np.linspace(0, 1, num=100), dims=['trigger_value'])
    # Initialize arrays to store scores
    #threshold_val=
    hit_rates = np.zeros_like(trigger_values)
    false_alarm_ratios = np.zeros_like(trigger_values)
    bias_scores = np.zeros_like(trigger_values)
    hanssen_kuipers_scores = np.zeros_like(trigger_values)
    heidke_skill_scores = np.zeros_like(trigger_values)
    #auroc_scores = np.zeros_like(trigger_values)
    #auroc_lb = np.zeros_like(trigger_values)
    #auroc_ub = np.zeros_like(trigger_values)
    for i, trigger_value in enumerate(trigger_values):
        # Calculate the dichotomous event for observation and forecast
        obs_event1 = obs_data <= threshold
        #obs_event1['name'] = 'observed_event' 
        obs_event1.name = 'observed_event' 

        drought_forecast_probablity= (ens_data <= threshold).mean(dim='member')

        #forecast_event1 = ens_data.mean(dim='member') >= trigger_value
        forecast_event1=drought_forecast_probablity>=trigger_value
        
        #forecast_event1['name'] = 'forecasted_event'
        forecast_event1.name = 'forecasted_event'

        # Create a 2D histogram using xhistogram
        contingency_table = xhist.histogram(
            #obs_event.stack(point=['time', 'lat', 'lon']),
            #forecast_event.stack(point=['time', 'lat', 'lon']),
            obs_event1,
            forecast_event1,
            bins=[2, 2],
            density=False
        )
    
        # Extract contingency table counts
        contingency_table = contingency_table.data
        correct_negatives = contingency_table[0, 0]
        false_alarms = contingency_table[0, 1]
        misses = contingency_table[1, 0]
        hits = contingency_table[1, 1]
    
        # Calculate scores
        total = hits + false_alarms + misses + correct_negatives
        hit_rates[i] = hits / (hits + misses) if (hits + misses) > 0 else np.nan
        false_alarm_ratios[i] = false_alarms / (false_alarms + hits) if (false_alarms + hits) > 0 else np.nan
        #false_alarm_ratios[i] = false_alarms / (false_alarms + correct_negatives) if (false_alarms + correct_negatives) > 0 else np.nan
        bias_scores[i] = (hits + false_alarms) / (hits + misses) if (hits + misses) > 0 else np.nan
        hanssen_kuipers_scores[i] = hit_rates[i] - false_alarm_ratios[i]
        heidke_skill_scores[i] = (hits * correct_negatives - misses * false_alarms) / total
        # calculate auroc
        auroc_bootstrap_scores = []
        n_bootstrap = 1000
        for _ in range(n_bootstrap):
            bootstrap_counts = np.random.multinomial(
                hits + misses + false_alarms + correct_negatives,
                [hits, misses, false_alarms, correct_negatives] / (hits + misses + false_alarms + correct_negatives),
                size=1
            )
            bootstrap_hits, bootstrap_misses, bootstrap_false_alarms, bootstrap_correct_negatives = bootstrap_counts[0]
            auroc_bootstrap_scores.append(calculate_auroc(bootstrap_hits, bootstrap_misses, bootstrap_false_alarms, bootstrap_correct_negatives))
    
        auroc_scores[i] = np.mean(auroc_bootstrap_scores)
        auroc_lb[i], auroc_ub[i] = np.percentile(auroc_bootstrap_scores, [2.5, 97.5])
        print(i)
    df = pd.DataFrame({
        'hit_rates': hit_rates,
        'false_alarm_ratios': false_alarm_ratios,
        'bias_scores': bias_scores,
        'hanssen_kuipers_scores': hanssen_kuipers_scores,
        'heidke_skill_scores': heidke_skill_scores,
        'auroc_scores': auroc_scores,
        'auroc_lb': auroc_lb,
        'auroc_ub': auroc_ub
    })
    df['trigger_values']=trigger_values
    df.insert(0, 'threshold', threshold)
    df.insert(0, 'trigger_values', trigger_values)
    df.to_csv(f'{data_path}{region_id}_{season_str}_{level}_lt{lead_int}.csv')
    return df


region_id=0
season_str='MAM'
level='mod'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
df=score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)




In [ ]:
## do for remaining 89 season/lead time/region combination 

In [ ]:
##############
##############
###region 0
#mod##################
region_id=0
season_str='MAM'
level='mod'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=0
season_str='MAM'
level='mod'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='mod'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='mod'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='mod'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#sev##################

region_id=0
season_str='MAM'
level='sev'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=0
season_str='MAM'
level='sev'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='sev'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='sev'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='sev'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#ext##################
region_id=0
season_str='MAM'
level='ext'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=0
season_str='MAM'
level='ext'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='ext'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='ext'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='MAM'
level='ext'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)
##############
##############
###region 1

region_id=1
season_str='MAM'
level='mod'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=1
season_str='MAM'
level='mod'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='mod'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='mod'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='mod'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#sev##################

region_id=1
season_str='MAM'
level='sev'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=1
season_str='MAM'
level='sev'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='sev'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='sev'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='sev'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#ext##################
region_id=1
season_str='MAM'
level='ext'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=1
season_str='MAM'
level='ext'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='ext'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='ext'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='MAM'
level='ext'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

##############
##############
###region 2

region_id=2
season_str='MAM'
level='mod'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=2
season_str='MAM'
level='mod'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='mod'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='mod'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='mod'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#sev##################

region_id=2
season_str='MAM'
level='sev'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=2
season_str='MAM'
level='sev'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='sev'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='sev'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='sev'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#ext##################
region_id=2
season_str='MAM'
level='ext'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=2
season_str='MAM'
level='ext'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='ext'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='ext'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='MAM'
level='ext'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#OND
##############
##############
###region 1

region_id=1
season_str='OND'
level='mod'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=1
season_str='OND'
level='mod'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='mod'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='mod'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='mod'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#sev##################

region_id=1
season_str='OND'
level='sev'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=1
season_str='OND'
level='sev'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='sev'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='sev'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='sev'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#ext##################
region_id=1
season_str='OND'
level='ext'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=1
season_str='OND'
level='ext'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='ext'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='ext'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=1
season_str='OND'
level='ext'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

##############
##############
###region 2

region_id=2
season_str='OND'
level='mod'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=2
season_str='OND'
level='mod'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='mod'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='mod'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='mod'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#sev##################

region_id=2
season_str='OND'
level='sev'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=2
season_str='OND'
level='sev'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='sev'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='sev'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='sev'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#ext##################
region_id=2
season_str='OND'
level='ext'
lead_int=1
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=2
season_str='OND'
level='ext'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='ext'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='ext'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=2
season_str='OND'
level='ext'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

###JJAS
#######

#JJAS
##############
##############
###region 0

region_id=0
season_str='JJAS'
level='mod'
lead_int=0
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=0
season_str='JJAS'
level='mod'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='mod'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='mod'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='mod'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#sev##################

region_id=0
season_str='JJAS'
level='sev'
lead_int=0
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=0
season_str='JJAS'
level='sev'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='sev'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='sev'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='sev'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

#ext##################
region_id=0
season_str='JJAS'
level='ext'
lead_int=0
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)


region_id=0
season_str='JJAS'
level='ext'
lead_int=2
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='ext'
lead_int=3
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='ext'
lead_int=4
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)

region_id=0
season_str='JJAS'
level='ext'
lead_int=5
obs_data, ens_data=make_obs_fct_dataset(region_id,season_str,lead_int)
score_gen(obs_data,ens_data,lead_int,region_id,season_str,level)